<a href="https://colab.research.google.com/github/isha-mg/ChatBot-using-Python/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

TASK - 1
ChatBot using Python

In [ ]:
import numpy as np
import json
import re
import tensorflow as tf
import random
import spacy
nlp = spacy.load('en_core_web_sm')

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
with open('/content/drive/MyDrive/Colab Notebooks/data_set1/Intent1.json') as f:
    intents = json.load(f)

In [28]:
def preprocessing(line):
    line = re.sub(r'[^a-zA-z.?!\']', ' ', line)
    line = re.sub(r'[ ]+', ' ', line)
    return line

In [29]:
inputs, targets = [], []
classes = []
intent_doc = {}

for intent in intents['intents']:
    if intent['intent'] not in classes:
        classes.append(intent['intent'])
    if intent['intent'] not in intent_doc:
        intent_doc[intent['intent']] = []
        
    for text in intent['text']:
        inputs.append(preprocessing(text))
        targets.append(intent['intent'])
        
    for response in intent['responses']:
        intent_doc[intent['intent']].append(response)

In [30]:
def tokenize_data(input_list):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='', oov_token='<unk>')
    
    tokenizer.fit_on_texts(input_list)
    
    input_seq = tokenizer.texts_to_sequences(input_list)

    input_seq = tf.keras.preprocessing.sequence.pad_sequences(input_seq, padding='pre')
    
    return tokenizer, input_seq

# preprocess input data
tokenizer, input_tensor = tokenize_data(inputs)

In [31]:
def create_categorical_target(targets):
    word={}
    categorical_target=[]
    counter=0
    for trg in targets:
        if trg not in word:
            word[trg]=counter
            counter+=1
        categorical_target.append(word[trg])
    
    categorical_tensor = tf.keras.utils.to_categorical(categorical_target, num_classes=len(word), dtype='int32')
    return categorical_tensor, dict((v,k) for k, v in word.items())

# preprocess output data
target_tensor, trg_index_word = create_categorical_target(targets)

In [32]:
print('input shape: {} and output shape: {}'.format(input_tensor.shape, target_tensor.shape))

input shape: (146, 9) and output shape: (146, 23)


In [34]:
epochs=50
vocab_size=len(tokenizer.word_index) + 1
embed_dim=512
units=128
target_length=target_tensor.shape[1]

In [35]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(vocab_size, embed_dim),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units, dropout=0.2)),
    tf.keras.layers.Dense(units, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(target_length, activation='softmax')
])

optimizer = tf.keras.optimizers.Adam(lr=1e-2)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, None, 512)         67584     
                                                                 
 bidirectional_2 (Bidirectio  (None, 256)              656384    
 nal)                                                            
                                                                 
 dense_4 (Dense)             (None, 128)               32896     
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_5 (Dense)             (None, 23)                2967      
                                                                 
Total params: 759,831
Trainable params: 759,831
Non-trainable params: 0
________________________________________________

In [36]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=4)

# train the model
model.fit(input_tensor, target_tensor, epochs=epochs, callbacks=[early_stop])

Epoch 1/50
5/5 [==============================] - 5s 79ms/step - loss: 3.1362 - accuracy: 0.0479
Epoch 2/50
5/5 [==============================] - 0s 68ms/step - loss: 3.0898 - accuracy: 0.1096
Epoch 3/50
5/5 [==============================] - 0s 67ms/step - loss: 3.0497 - accuracy: 0.1849
Epoch 4/50
5/5 [==============================] - 0s 66ms/step - loss: 2.9735 - accuracy: 0.2055
Epoch 5/50
5/5 [==============================] - 0s 64ms/step - loss: 2.8628 - accuracy: 0.2466
Epoch 6/50
5/5 [==============================] - 0s 66ms/step - loss: 2.7546 - accuracy: 0.3014
Epoch 7/50
5/5 [==============================] - 0s 64ms/step - loss: 2.5857 - accuracy: 0.3288
Epoch 8/50
5/5 [==============================] - 0s 64ms/step - loss: 2.3793 - accuracy: 0.3288
Epoch 9/50
5/5 [==============================] - 0s 65ms/step - loss: 2.1205 - accuracy: 0.3904
Epoch 10/50
5/5 [==============================] - 0s 67ms/step - loss: 1.9464 - accuracy: 0.4452
Epoch 11/50
5/5 [============

In [37]:
def response(sentence):
    sent_seq = []
    doc = nlp(repr(sentence))
    
    # split the input sentences into words
    for token in doc:
        if token.text in tokenizer.word_index:
            sent_seq.append(tokenizer.word_index[token.text])

        # handle the unknown words error
        else:
            sent_seq.append(tokenizer.word_index['<unk>'])

    sent_seq = tf.expand_dims(sent_seq, 0)
    # predict the category of input sentences
    pred = model(sent_seq)

    pred_class = np.argmax(pred.numpy(), axis=1)
    
    # choice a random response for predicted sentence
    return random.choice(intent_doc[trg_index_word[pred_class[0]]]), trg_index_word[pred_class[0]]

# chat with bot
print("Note: Enter 'quit' to break the loop.")
while True:
    input_ = input('You: ')
    if input_.lower() == 'quit':
        break
    res, typ = response(input_)
    print('Bot: {} -- TYPE: {}'.format(res, typ))
    print()

Note: Enter 'quit' to break the loop.
You: hola
Bot: Hi human, please tell me your GeniSys user -- TYPE: Greeting

You: how are you ?
Bot: Hi, how are you? I am great thanks! Please tell me your GeniSys user -- TYPE: CourtesyGreeting

You: what is your name?
Bot: GeniSys -- TYPE: RealNameQuery

You: can you tell me if you are self aware
Bot: That depends, can you prove that you are? -- TYPE: SelfAware

You: tell me a joke
Bot: Waiter, what is this stuff? That's bean salad sir. I know what it's been, but what is it now? -- TYPE: Jokes

You: what is your favourite movie
Bot: My favourite movie is 3 Idiots -- TYPE: Movie

You: can you make a tea
Bot: That depends, can you prove that you are? -- TYPE: SelfAware

You: quit
